## Extraction of Data from iSDA Datasets (GEE) ##

In [7]:
# Import Google Earth Engine API and Initialize it. 
import ee

ee.Authenticate()
ee.Initialize(project="ey-data-and-ai-challenge")


In [9]:
# Other import statements to be added as needed

import pandas as pd
import numpy as np

In [23]:
# Read coordinates from water quality training dataset, drop given features and date since iSDA datasets are static, i.e. not date dependent.

wq_df = pd.read_csv('../data/water_quality_training_dataset.csv')
wq_df = wq_df.drop(columns=['Total Alkalinity', 'Electrical Conductance', 'Dissolved Reactive Phosphorus', 'Sample Date'])
wq_df['id'] = wq_df.index
wq_df.head()


,Latitude,Longitude,id
0,-28.760833,17.730278,0
1,-26.861111,28.884722,1
2,-26.450000,28.085833,2
3,-27.671111,27.236944,3
4,-27.356667,27.286389,4


In [27]:
# Convert Coordintaes to ee.Features

features = []

for index, row in wq_df.iterrows():
    feat = ee.Feature(
        ee.Geometry.Point([row['Longitude'], row['Latitude']]),
        {'id': row['id']}
    )
    features.append(feat)

Read Google Earth Engine iSDA datasets (image) for extraction with specified coordinate bounds

In [22]:
# Reading all wanted iSDA datasets as EE images into a single image using ee.image.cat for quicker processing.

pH_img = ee.Image("ISDASOIL/Africa/v1/ph").select('mean_0_20').rename('pH')
phosphorous_img = ee.Image("ISDASOIL/Africa/v1/phosphorus_extractable").select('mean_0_20').rename('phosphorous')
cec_img = ee.Image("ISDASOIL/Africa/v1/cation_exchange_capacity").select('mean_0_20').rename('cec') # Cation Exchange Capacity
clay_img = ee.Image("ISDASOIL/Africa/v1/clay_content").select('mean_0_20').rename('clay') # clay content

# concatenate all images into a single image to query coordinates with

soil_img = ee.Image.cat([pH_img, phosphorous_img, cec_img, clay_img])



Create a Feature Collection from coordinates specified in wq_df and combined images in soil_img

In [28]:
fc = ee.FeatureCollection(features)